05-01-2023 CR:
this notebook is based on the cc_order.ipynb file in the specsv1-xx file retrieved 19-12-2022. 
the notebook is being adapted to accomodate for importing the provided SelleckChem library file and exporting an iDot file.
the latter part of the notebook is based on codes by Andreina.

In [23]:
import os.path
import time
import pandas as pd
import glob
import numpy as np
import csv 
from datetime import datetime
import re 
import os

# Settings to display more columns and rows
pd.set_option("max_colwidth", 200)
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 10)

print(os.getcwd())
# Find the input directory using glob
input_dirs = glob.glob('../*input*/')
if input_dirs:
	os.chdir(input_dirs[0])
else:
	print("Warning: No input directory found matching '../input*/'")
print(os.getcwd())

/share/data/analyses/christa/colopaint3D/colo8-input
/share/data/analyses/christa/colopaint3D/colo8-input


In [24]:
# Remove the zero before the well number
def strip_zeros(well):
    number = re.split('(\d+)', well)[1].lstrip('0')
    letter = re.split('(\d+)', well)[0]
    new_well = letter + number
    return new_well


# Remove extra white spaces that I entered by accident in plaid
def strip_spaces(a_str_with_spaces):
    # https://stackoverflow.com/questions/43332057/pandas-strip-white-space
    return a_str_with_spaces.replace(' ', '')

In [25]:
OutputDir = "echo-protocols" # Where do you want to save the Echo protocols? 
if not os.path.exists(OutputDir):
    os.makedirs(OutputDir)


SupportDir = "support-files" # Where do you want to save the merged data csv? 
if not os.path.exists(SupportDir):
    os.makedirs(SupportDir)


ImportDir = "import-files"

In [26]:
## Split the protocol in water and dsmo parts

# solvent = 'water'
solvent = 'dmso'

In [27]:

# # Add some echo settings

max_volume_uL = 70 # allow for a bit of variation


### Import PLAID files, combine, and fill in missing values

In [28]:
#
# some conventions
#

# DMSO and water negative controls always in %
# Compound vol always in nL
# Well volume always in uL
# Stock conc always in mM

In [ ]:
#
# Import all plaid files, do some cleaning, and combine them
#

# experiment code
# exp_name = 'colo8-v1-VP-organoid-48h-P1'


# DMSO percentage that you allow in experiment
dmso_max_perc = 0.1

# H20 percentage that you allow in experiment (at times cmpds are dissolved in water)
h2o_max_perc = 5 # arbitrary, but I assume that water is not harmful like dmso. Also there is a control.

# Working volume in wells in ul
well_vol_uL = 40

plaid_folder = 'plaid_files'


# Spot multiple plates to one plate copy.
duplication_factor = 1 # For me specifically it means that I will take 25µl in the plate copy plate. 


# Combine all plaid-csv-files from specified folder into a dataframe, replace plateID "plate_1" in each file into plate_1 - plate_X
filelist = glob.glob(os.path.join(plaid_folder, '*.csv'))
df_combined = pd.DataFrame()
for idx, file in enumerate(sorted(filelist)):
    # read csv file and replace plateID plate_1 in all files to xxx_l1_p1
    df = pd.read_csv(file,index_col=False,converters={'cmpdnum': strip_spaces}, usecols=['plateID', 'well', 'cmpdname', 'CONCuM', 'cmpdnum'])
    # new_plate_name = 'plate_' + str(idx + 1)
    new_plate_name = os.path.splitext(os.path.basename(file))[0] # assign the barcode instead
    df['plateID'] = df['plateID'].str.replace('plate_1', new_plate_name)
    
    df_combined = pd.concat([df_combined, df], ignore_index=True)    

# drop all rows that have the '------' id
df_combined = df_combined[df_combined["plateID"].str.contains("----------") == False]

exp_name = new_plate_name

print(f"✓ Loaded {len(df_combined)} wells from PLAID")
print(f"  Unique compounds: {df_combined['cmpdname'].nunique()}")
print(f"  Plates: {df_combined['plateID'].nunique()}")


✓ Loaded 308 wells from PLAID
  Unique compounds: 12
  Plates: 1


### Load the SelleckChem library, assign compounds, and find optimal stocks

In [30]:
#
# Add well volumes
#

# add columns with default values
df_combined['well_vol_uL'] = well_vol_uL

# adjust the well notations
df_combined['well_w_zero'] = df_combined['well']
df_combined['well'] = df_combined['well'].map(strip_zeros)

df_combined['well_letter'] = df_combined['well'].str.extract(r'([A-Z])')
df_combined['well_number'] = df_combined['well'].str.extract(r'(\d+)')


In [31]:
# 
# Prepare well notations (no column adjustment needed)
#

df_combined = df_combined.astype({'well_number': int})

df_combined['well'] = df_combined['well_letter'] + df_combined['well_number'].astype(str)
df_combined['well_w_zero'] = df_combined['well_letter'] + df_combined['well_number'].astype(str).apply(lambda x: x.zfill(2))
df_combined['well_number'] = df_combined['well_number'].astype(str).apply(lambda x: x.zfill(2))

In [32]:
#
# Import colo8 compound library
#

library_file = 'colo8-list'
df_medchem = pd.read_csv("{}/{}.csv".format(ImportDir,library_file),index_col=False)

display(df_medchem)

,cmpd-code,ProductName,Plate Location,Concentration,Solvent,Pathway
0,colo-002,veliparib,b2,10 mM,dmso,DNA Damage
1,colo-009,olaparib,a3,10 mM,dmso,DNA Damage
2,colo-020,fluorouracil,d4,10 mM,dmso,DNA Damage
3,colo-028,gemcitabine,d5,10 mM,dmso,DNA Damage
4,colo-029,trifluridine,e5,10 mM,dmso,DNA Damage
...,...,...,...,...,...,...
7,colo-044,abemaciclib,d7,10 mM,dmso,Cell Cycle
8,ref-001,etoposide,c4,10 mM,dmso,NaN
9,ref-002,fenbendazole,e4,10 mM,dmso,NaN
10,ref-003,berberine chloride,f4,10 mM,dmso,NaN


In [33]:
#
# Assign the compound names
# 

# Map only the compounds that are in the library, leave the rest as is
df_combined['cmpdname'] = df_combined['cmpdname'].map(df_medchem.set_index('cmpd-code')['ProductName']).fillna(df_combined['cmpdname'])


In [34]:
## Export the file (no combinations in colo8)
plaid_combined_filename = 'colo8-plates'
df_combined.to_csv("{}/{}.csv".format(SupportDir,plaid_combined_filename), index=False, sep=",")

In [35]:
# #
# # Clean up the data
# # 

df_library = df_medchem.copy()

# Parse concentration and stock_unit from Concentration column (format: "10 mM")
df_library['stock_unit'] = df_library['Concentration'].str.split('(\d+\.\d+|\d+)', expand=True)[2]
df_library['max_stock'] = df_library['Concentration'].str.extract(r'(\d+\.\d+|\d+)').astype('float')

# Solvent is already lowercase in colo8-list.csv ('dmso' or 'water')
df_library['solvent'] = df_library['Solvent']

# prepare for stockfinder by adding solvent and max_stock columns to df_combined
df_forstockfinder = df_combined.merge(df_library[['max_stock', 'solvent', 'stock_unit', 'ProductName']], 
    left_on = 'cmpdname',
    right_on = 'ProductName',
    how = 'left'
    )


In [36]:
#
# Calculate stock conc v2 - idot from selleck  
#

# Stock conc function
def stockfinder_idot(x_uM,max_stock,solvent, stock_unit):
    
    #  ----------------------------------------------------------------------------------------------------------------------------
    #     #  DESCRIPTION
    #     #  v1 * c1 = v2 * c2
    #     #  volume stock solution (nl) * concentration stock solution (mM)  = volume destination (ul) * concentration final (uM)
    #     #
    #     #  c1 = TO BE CALCULATED
    #     #  v1 = volume of stock in ul (minimum is 8 and maximum is dependent on dmso limits)
    #     #  c2 = target concentration in uM "CONCuM" value
    #     #  v2 = working_volume_ul (e.g. 40 ul)
    #     #
    #     #  c1_max (highest stock concentration that can be used for target concentration)
    #     #  c1_min (lowest stock concentration that can be used for target concentration)
    #  ----------------------------------------------------------------------------------------------------------------------------

    # check if 'dmso' or 'water' (notation is SelleckChem's)
    max_perc = []
    if solvent == 'dmso':
        max_perc = dmso_max_perc
    elif solvent == 'water':
        max_perc = h2o_max_perc
    else:
        max_perc = 0.001 # if there is an unknown solvent, put the stock_conc to 0 

    # set available stock concentrations
    availstocks_mM = list([max_stock, 1.0, 0.1, 0.01, 0.001, 0.0001]) # think about whether it really needs to be a list # sort them from highest to lowest for later

  

    # minimum volume to dispense
    minV1_nl = 2.5
     # maximum volume to be dispensed while staying below dmso limits            
    maxV1_nl = (max_perc / 100) * (well_vol_uL*1000)                                              

    stock_unit = stock_unit.strip()

    if stock_unit == 'mM':
        # calculate  lowest and highest stock that can be used
        c1_low = (well_vol_uL * x_uM) / maxV1_nl                                  
        c1_high = (well_vol_uL * x_uM) / minV1_nl                                
    elif stock_unit == '%':
        x_uM = x_uM * 1000 # convert
        # calculate  lowest and highest stock that can be used
        c1_low = 100  # always the lowest available stock %                            
        c1_high = 100 # in % always the highest available stock %
        #TODO: something is weird here, think about it clearly
    elif stock_unit == 'mg/mL':
         # set available stock concentrations
        availstocks_mM = list([max_stock, 5,0, 2.5, 1.0, 0.1, 0.01, 0.001, 0.0001]) # think about whether it really needs to be a list # sort them from highest to lowest for later
        # calculate  lowest and highest stock that can be used
        c1_low = (well_vol_uL * x_uM) / maxV1_nl  # always the lowest available stock %                            
        c1_high = (well_vol_uL * x_uM) / minV1_nl # in % always the highest available stock %
        #TODO: something is weird here, think about it clearly
    else: 
        # print('unknown stock unit')
        return [None, None]  

    # Find possible stock concentrations from available
    possible_stocks = [i for i in availstocks_mM if i >= c1_low and i <= c1_high]

    # Filter stocks based on volume requirements with 2.5 nL rounding
    # Prefer stocks that give volumes close to 2.5 nL multiples (minimize rounding error)
    valid_stocks = []
    for stock in possible_stocks:
        volume_nL = (well_vol_uL * x_uM) / stock
        rounded_vol = round(volume_nL / 2.5) * 2.5  # Round to nearest 2.5 nL
        
        # Check if rounded volume is within acceptable range
        if rounded_vol >= minV1_nl and rounded_vol <= maxV1_nl:
            # Calculate rounding error as percentage
            error_pct = abs(volume_nL - rounded_vol) / volume_nL * 100 if volume_nL > 0 else 0
            valid_stocks.append((stock, error_pct))
    
    # Select the highest stock with acceptable rounding error (prefer <5% error)
    if not valid_stocks:
        return [None, None]
    else:
        # Sort by error (ascending), then by stock concentration (descending)
        valid_stocks.sort(key=lambda x: (x[1], -x[0]))
        highest_stock = valid_stocks[0][0]
        return [highest_stock, availstocks_mM]

#
#
# Add new column with result from applying stock_concentration calculation function
#
#

df_with_stock = df_forstockfinder.copy()
df_with_stock['CONCuM'] = df_with_stock['CONCuM'].astype('float')
df_with_stock['max_stock'] = df_with_stock['max_stock'].astype('float')

# Replace NaN values in 'stock_unit' column with an empty string
df_with_stock['stock_unit'] = df_with_stock['stock_unit'].fillna('')

# Apply the stockfinder function using information from three columns
df_with_stock[['stock_conc_mM', 'availstocks_mM']] =  df_with_stock.apply(lambda x: stockfinder_idot(x.CONCuM, x.max_stock, x.solvent, x.stock_unit), axis=1, result_type='expand')
df_with_stock = df_with_stock.drop(columns='max_stock')
df_with_stock['cmpd_w_stock'] = df_with_stock['cmpdname'] + "[" + df_with_stock['stock_conc_mM'].astype(str)+ "]"


#
# Print Warning and Display all rows with zero values
#
df_stock_conc_zero = df_with_stock[df_with_stock['stock_conc_mM'].isnull()]
if not df_stock_conc_zero.empty:
    print("Warning - following rows could not be assigned a Stock Concentration")
    display(df_stock_conc_zero)

### Run some tests and calculate backfills

In [37]:
#
# Calculate compound volumes and adjust concentrations for 2.5 nL increments
#

# Copy last cells df
df_w_cmpd = df_with_stock.copy()

# Convert column types
df_w_cmpd['CONCuM'] = df_w_cmpd['CONCuM'].astype(float)
df_w_cmpd['well_vol_uL'] = df_w_cmpd['well_vol_uL'].astype(float)
df_w_cmpd['stock_conc_mM'] = df_w_cmpd['stock_conc_mM'].astype(float)

# Store original concentrations for tracking
df_w_cmpd['CONCuM_original'] = df_w_cmpd['CONCuM'].copy()

# Calculate compound/dmso volumes
# We split df in two because DMSO is calculated in % other in uM
# dmso, water, and water+ concentration in field CONCuM is % and not uM

# compounds
df_w_cmpd.loc[df_w_cmpd['stock_unit'] == ' mM', 'CompVol_nL'] = (df_w_cmpd['CONCuM'] * df_w_cmpd['well_vol_uL']) / df_w_cmpd['stock_conc_mM']

# compounds
df_w_cmpd.loc[df_w_cmpd['stock_unit'] == ' mg/mL', 'CompVol_nL'] = (df_w_cmpd['CONCuM'] * df_w_cmpd['well_vol_uL']) / df_w_cmpd['stock_conc_mM']

# dmso / water conc is in % even although column name is CONCuM
df_w_cmpd.loc[df_w_cmpd['stock_unit'] == ' %', 'CompVol_nL'] = (df_w_cmpd['well_vol_uL'] * 1000) * (df_w_cmpd['CONCuM'] / 100)

# CRITICAL: Round all volumes to nearest 2.5 nL (Echo requirement)
# and back-calculate adjusted concentrations
df_w_cmpd['CompVol_nL_unrounded'] = df_w_cmpd['CompVol_nL'].copy()
df_w_cmpd['CompVol_nL'] = (df_w_cmpd['CompVol_nL'] / 2.5).round() * 2.5

# Back-calculate actual achieved concentrations based on rounded volumes
# For mM compounds: CONCuM = (CompVol_nL * stock_conc_mM) / well_vol_uL
df_w_cmpd.loc[df_w_cmpd['stock_unit'] == ' mM', 'CONCuM'] = \
    (df_w_cmpd['CompVol_nL'] * df_w_cmpd['stock_conc_mM']) / df_w_cmpd['well_vol_uL']

df_w_cmpd.loc[df_w_cmpd['stock_unit'] == ' mg/mL', 'CONCuM'] = \
    (df_w_cmpd['CompVol_nL'] * df_w_cmpd['stock_conc_mM']) / df_w_cmpd['well_vol_uL']

# For % compounds: CONCuM = (CompVol_nL / (well_vol_uL * 1000)) * 100
df_w_cmpd.loc[df_w_cmpd['stock_unit'] == ' %', 'CONCuM'] = \
    (df_w_cmpd['CompVol_nL'] / (df_w_cmpd['well_vol_uL'] * 1000)) * 100

# Transfer Volume is in µl
df_w_cmpd['CompVol_uL'] = df_w_cmpd['CompVol_nL'] / 1000

# Report adjustments
df_w_cmpd['conc_change_pct'] = ((df_w_cmpd['CONCuM'] - df_w_cmpd['CONCuM_original']) / df_w_cmpd['CONCuM_original'] * 100).round(2)
df_adjusted = df_w_cmpd[df_w_cmpd['CompVol_nL'] != df_w_cmpd['CompVol_nL_unrounded']].copy()

if len(df_adjusted) > 0:
    print(f"⚠ Adjusted {len(df_adjusted)} concentrations to ensure volumes are divisible by 2.5 nL")
    print(f"  Max concentration change: {df_adjusted['conc_change_pct'].abs().max():.2f}%")
    print(f"  Mean concentration change: {df_adjusted['conc_change_pct'].abs().mean():.2f}%")
    print(f"\nSample adjustments:")
    print(df_adjusted[['cmpdname', 'CONCuM_original', 'CONCuM', 'CompVol_nL_unrounded', 'CompVol_nL']].head(10).to_string(index=False))
else:
    print("✓ All volumes were already divisible by 2.5 nL")

# Clean up tracking columns
df_w_cmpd = df_w_cmpd.drop(columns=['CONCuM_original', 'CompVol_nL_unrounded', 'conc_change_pct'])

⚠ Adjusted 80 concentrations to ensure volumes are divisible by 2.5 nL
  Max concentration change: 4.17%
  Mean concentration change: 4.17%

Sample adjustments:
    cmpdname  CONCuM_original  CONCuM  CompVol_nL_unrounded  CompVol_nL
trifluridine              3.0   3.125                  12.0        12.5
    olaparib              3.0   3.125                  12.0        12.5
fluorouracil              3.0   3.125                  12.0        12.5
fluorouracil              3.0   3.125                  12.0        12.5
   veliparib              3.0   3.125                  12.0        12.5
 gemcitabine              3.0   3.125                  12.0        12.5
 abemaciclib              3.0   3.125                  12.0        12.5
 binimetinib              3.0   3.125                  12.0        12.5
       sn-38              3.0   3.125                  12.0        12.5
    olaparib              3.0   3.125                  12.0        12.5


In [38]:
print(f"✓ Calculated compound volumes for {len(df_w_cmpd)} wells")
print(f"  Sample of calculated volumes:")
display(df_w_cmpd[['well', 'cmpdname', 'CONCuM', 'stock_conc_mM', 'CompVol_nL']].head(10))

✓ Calculated compound volumes for 308 wells
  Sample of calculated volumes:


,well,cmpdname,CONCuM,stock_conc_mM,CompVol_nL
0,B2,trifluridine,3.125,10.0,12.5
1,B3,etoposide,2.500,10.0,10.0
2,B4,olaparib,3.125,10.0,12.5
3,B5,binimetinib,10.000,10.0,40.0
4,B6,gemcitabine,1.000,1.0,40.0
5,B7,fluorouracil,1.000,1.0,40.0
6,B8,abemaciclib,1.000,1.0,40.0
7,B9,fluorouracil,3.125,10.0,12.5
8,B10,veliparib,1.000,1.0,40.0
9,B11,berberine chloride,2.500,10.0,10.0


In [39]:
#
# Dynamically generate source plates based on stocks selected by stockfinder
# Group by solvent to create separate source plates
# Organize by: treatment compounds (grouped by name), then reference compounds, then solvent
#

print("="*70)
print("GENERATING SOURCE PLATES DYNAMICALLY")
print("="*70)

# Collect unique (compound, stock_concentration, solvent) combinations
df_stocks_needed = df_w_cmpd[['cmpdname', 'stock_conc_mM', 'solvent', 'stock_unit', 'cmpdnum']].copy()
df_stocks_needed = df_stocks_needed.dropna(subset=['stock_conc_mM'])  # Remove any NaN stocks

# Extract base compound code for categorization (before dropping duplicates)
df_stocks_needed['compound_code'] = df_stocks_needed['cmpdnum'].str.split('_').str[0]

# Drop duplicates based on compound name, stock concentration, and solvent only
# (not cmpdnum, which can vary for the same compound at different target concentrations)
df_stocks_needed = df_stocks_needed.drop_duplicates(subset=['cmpdname', 'stock_conc_mM', 'solvent'])

# Group by solvent
solvents = df_stocks_needed['solvent'].unique()

for solvent_name in solvents:
    if pd.isna(solvent_name):
        continue
        
    print(f"\n--- Generating source plate for solvent: {solvent_name} ---")
    
    # Get all stocks for this solvent
    df_solvent_stocks = df_stocks_needed[df_stocks_needed['solvent'] == solvent_name].copy()
    
    # Categorize compounds
    treatment_mask = ~df_solvent_stocks['compound_code'].str.startswith('ref-') & (df_solvent_stocks['compound_code'] != 'dmso')
    reference_mask = df_solvent_stocks['compound_code'].str.startswith('ref-')
    solvent_mask = df_solvent_stocks['compound_code'] == 'dmso'
    
    df_treatments = df_solvent_stocks[treatment_mask].copy()
    df_references = df_solvent_stocks[reference_mask].copy()
    df_solvents = df_solvent_stocks[solvent_mask].copy()
    
    # Sort treatments: by compound name, then by stock concentration (descending)
    df_treatments = df_treatments.sort_values(['cmpdname', 'stock_conc_mM'], ascending=[True, False])
    
    # Sort references: by compound name, then by stock concentration (descending)
    df_references = df_references.sort_values(['cmpdname', 'stock_conc_mM'], ascending=[True, False])
    
    # Combine in order: treatments, references
    df_ordered = pd.concat([df_treatments, df_references], ignore_index=True)
    
    # Create source plate layout with column-wise organization
    source_rows = []
    
    # Track current column and row
    current_col = 1
    current_row = 0  # 0=A, 1=B, 2=C, etc.
    prev_compound = None
    
    for idx, row in df_ordered.iterrows():
        compound = row['cmpdname']
        stock_conc = row['stock_conc_mM']
        stock_unit = row['stock_unit'].strip()
        
        # If this is a different compound, move to next column and reset row
        if prev_compound is not None and compound != prev_compound:
            current_col += 1
            current_row = 0
        
        # Generate well location
        well_letter = chr(ord('A') + current_row)  # A, B, C, D, ...
        well_number = current_col
        well_padded = f"{well_letter}{str(well_number).zfill(2)}"
        well_simple = f"{well_letter}{well_number}"
        
        source_rows.append({
            'sourceID': f'source_{solvent_name}',
            'well_original': well_padded,
            'well_source': well_padded,
            'Compound': compound,
            'CONCmM': stock_conc,
            'well_letter': well_letter,
            'well_number': well_number,
            'well': well_simple
        })
        
        print(f"  {well_simple:4s}: {compound:22s} [{stock_conc} {stock_unit}]")
        
        current_row += 1
        prev_compound = compound
    
    # Add the solvent control at the end (next column)
    if solvent_name == 'dmso':
        control_conc = 100.0  # 100% for DMSO
    elif solvent_name == 'water':
        control_conc = 100.0  # 100% for water
    else:
        control_conc = 100.0
    
    current_col += 1
    well_letter = 'A'
    well_number = current_col
    well_padded = f"{well_letter}{str(well_number).zfill(2)}"
    well_simple = f"{well_letter}{well_number}"
    
    source_rows.append({
        'sourceID': f'source_{solvent_name}',
        'well_original': well_padded,
        'well_source': well_padded,
        'Compound': solvent_name,
        'CONCmM': control_conc,
        'well_letter': well_letter,
        'well_number': well_number,
        'well': well_simple
    })
    
    print(f"  {well_simple:4s}: {solvent_name:22s} [{control_conc} %] (solvent control)")
    
    # Create DataFrame
    df_source_plate = pd.DataFrame(source_rows)
    
    # Save to file
    source_filename = f"{SupportDir}/colo8-SOURCE-{solvent_name}.csv"
    df_source_plate.to_csv(source_filename, index=False)
    print(f"\n✓ Saved: {source_filename}")
    print(f"  Total wells: {len(source_rows)}")

print("\n" + "="*70)
print("SOURCE PLATE GENERATION COMPLETE")
print("="*70)

GENERATING SOURCE PLATES DYNAMICALLY

--- Generating source plate for solvent: dmso ---
  A1  : abemaciclib            [10.0 mM]
  B1  : abemaciclib            [1.0 mM]
  A2  : binimetinib            [10.0 mM]
  B2  : binimetinib            [1.0 mM]
  A3  : fluorouracil           [10.0 mM]
  B3  : fluorouracil           [1.0 mM]
  A4  : gemcitabine            [10.0 mM]
  B4  : gemcitabine            [1.0 mM]
  A5  : olaparib               [10.0 mM]
  B5  : olaparib               [1.0 mM]
  A6  : sn-38                  [10.0 mM]
  B6  : sn-38                  [1.0 mM]
  A7  : trifluridine           [10.0 mM]
  B7  : trifluridine           [1.0 mM]
  A8  : veliparib              [10.0 mM]
  B8  : veliparib              [1.0 mM]
  A9  : berberine chloride     [10.0 mM]
  A10 : etoposide              [10.0 mM]
  A11 : fenbendazole           [10.0 mM]
  A12 : dmso                   [100.0 %] (solvent control)

✓ Saved: support-files/colo8-SOURCE-dmso.csv
  Total wells: 20

SOURCE PLATE GENE

In [40]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Assign a unique color per compound using Plotly's built-in palette
unique_compounds = df_w_cmpd['ProductName'].unique()
color_palette = px.colors.qualitative.Set3  # Using a categorical color palette
color_dict = {compound: color_palette[i % len(color_palette)] for i, compound in enumerate(unique_compounds)}

# Adjust position if multiple compounds are in the same well
compound_counts = df_w_cmpd.groupby(['well_number', 'well_letter']).size()
compound_offsets = {}

for (well_x, well_y), count in compound_counts.items():
    if count > 1:
        offsets = np.linspace(-0.2, 0.2, count)  # Spread dots in X direction
        compound_offsets[(well_x, well_y)] = offsets
    else:
        compound_offsets[(well_x, well_y)] = [0]  # No offset needed

# Track which compounds have been added to legend
legend_added = set()

# Create scatter plot traces
scatter_traces = []
for index, row in df_w_cmpd.iterrows():
    well_x, well_y = row['well_number'], row['well_letter']
    compound_name = row['ProductName']
    
    # Get color for compound
    color = color_dict.get(compound_name, "gray")
    
    # Get the offset position for this compound
    offset_index = list(df_w_cmpd[(df_w_cmpd['well_number'] == well_x) & 
                                  (df_w_cmpd['well_letter'] == well_y)].index).index(index)
    x_adjusted = int(well_x) + compound_offsets[(well_x, well_y)][offset_index]
    
    # Only show in legend if this compound hasn't been added yet
    show_in_legend = compound_name not in legend_added
    if show_in_legend:
        legend_added.add(compound_name)
    
    # Add a scatter trace for each compound
    scatter_traces.append(
        go.Scatter(
            x=[x_adjusted],
            y=[well_y],
            mode="markers",
            marker=dict(size=10, color=color, line=dict(width=1, color="black")),
            name=compound_name,
            showlegend=show_in_legend,
            hoverinfo="text",
            text=f"Compound: {compound_name}<br>Well: {well_y}{well_x}<br>Concentration: {row['CONCuM']} uM"
        )
    )


# Create the figure
fig = go.Figure()
# Adjust the size of the figure
fig.update_layout(width=1600, height=800)

# Add all scatter traces to the figure
for trace in scatter_traces:
    fig.add_trace(trace)


# Customize layout
fig.update_layout(
    title="Interactive Plate Map with Different Compounds",
    xaxis=dict(title="Well Number", tickmode="array", tickvals=np.arange(1, 25)),
    yaxis=dict(title="Well Letter", tickmode="array", tickvals=sorted(df_w_cmpd['well_letter'].unique())),
    legend_title="Compounds",
    showlegend=True,
    template="plotly_white"
)

# Invert the y-axis to match the plate layout
fig.update_yaxes(autorange="reversed")

# Sort the y-axis categories alphabetically
fig.update_yaxes(categoryorder="array", categoryarray=sorted(df_w_cmpd['well_letter'].unique()))

# Save as html
output_file = "{}/{}.html".format(OutputDir, exp_name)
fig.write_html(output_file)
print(f"✓ Saved plate visualization: {output_file}")

✓ Saved plate visualization: echo-protocols/colo8-v1-VP-organoid-48h-P1-L1.html


In [41]:
# More statistics

number_of_replicates = df_w_cmpd.groupby(['cmpdname'])['cmpdname'].count()
print("Number of replicates per_compound_and_stock")
with pd.option_context('display.max_rows', 10, 'display.min_rows', 30):
        display(number_of_replicates)

Number of replicates per_compound_and_stock


cmpdname
abemaciclib           30
berberine chloride    10
binimetinib           30
dmso                  38
etoposide             10
                      ..
gemcitabine           30
olaparib              30
sn-38                 30
trifluridine          30
veliparib             30
Name: cmpdname, Length: 12, dtype: int64

In [42]:
number_of_replicates

cmpdname
abemaciclib           30
berberine chloride    10
binimetinib           30
dmso                  38
etoposide             10
                      ..
gemcitabine           30
olaparib              30
sn-38                 30
trifluridine          30
veliparib             30
Name: cmpdname, Length: 12, dtype: int64

In [43]:
# 
# Calculate backfill
#

if 'backfill_to_vol_uL' in locals():
    del backfill_to_vol_uL
else:
    print('')

# User defined volume in nL to backfill to (if this is set it will override calculated value)
# Comment out value if not used
# backfill_to_vol_uL = 0.04

df_backfill = df_w_cmpd.copy()

# calculate max DMSO that is allowed in well (before toxic, calculated from well_vol_uL and dmso_max_perc)
maxDMSO = (dmso_max_perc / 100) * (well_vol_uL) # in ul --> also use this value for water


if 'backfill_to_vol_uL' in locals():
    target_vol_DMSO_per_well = backfill_to_vol_uL
    print("User defined max volume to backfill to: " + str(backfill_to_vol_uL))
else:
    target_vol_DMSO_per_well = maxDMSO
    print("Max allowed DMSO in well (before toxic, calculated from well_vol_uL and dmso_max_perc) (nL): " + str(maxDMSO))
print('')    

# Group by well and sum the compound volumes 
df_backfill = df_backfill.groupby(['well']).agg({'CompVol_uL': 'sum'}).reset_index()

# Only backfill when the solvent is dmso
df_backfill = df_backfill.merge(df_w_cmpd[['well', 'solvent']], on='well', how='left')
df_backfill = df_backfill[df_backfill['solvent'] == 'dmso']

df_backfill['backfill_vol_uL'] = target_vol_DMSO_per_well - df_backfill['CompVol_uL']



# Warn if backfill is negative
df_backfill_negative = df_backfill[df_backfill.backfill_vol_uL < 0]
if(df_backfill_negative.empty):
    print("OK, no negative backfill found")
else:
     print("WARNING! Negative backfill results found")
     print("They will be set to 0")
     display(df_backfill_negative)

# Set backfill to 0 if negative  
df_backfill.loc[df_backfill ['backfill_vol_uL'] < 0, 'backfill_vol_uL'] = 0


# # Add a column with solvent + stock (this bit will only work once)
# df_w_cmpd = df_w_cmpd.merge(df_controls_refcmpds[['max_stock', 'cmpdname']], left_on='solvent', right_on='cmpdname').drop('cmpdname_y', axis=1)
# df_w_cmpd.rename(columns = {'cmpdname_x':'cmpdname', 'max_stock':'solvent_stock'}, inplace = True)
# df_w_cmpd['solvent_w_stock'] = df_w_cmpd['solvent'] + "[" + (df_w_cmpd['solvent_stock']).astype(str) + "]"


# with pd.option_context('display.max_rows', 20, 'display.min_rows', 20):
#         display(df_w_cmpd)


Max allowed DMSO in well (before toxic, calculated from well_vol_uL and dmso_max_perc) (nL): 0.04

OK, no negative backfill found


In [ ]:
df_backfill = df_backfill.query('backfill_vol_uL > 0')
df_backfill.rename(columns = { 'backfill_vol_uL' : 'CompVol_nL'}, inplace = True)
df_backfill['CompVol_nL'] = df_backfill['CompVol_nL'].apply(lambda x: x * 1000) # Transfer Volume is in nl

# CRITICAL: Round backfill volumes to nearest 2.5 nL (Echo requirement)
df_backfill['CompVol_nL'] = (df_backfill['CompVol_nL'] / 2.5).round() * 2.5

df_backfill.drop('CompVol_uL', axis=1, inplace=True)    
df_backfill['cmpdname'] = 'dmso'
df_backfill['cmpd_w_stock'] = 'dmso[100.0]'
df_backfill['plateID'] = exp_name



# Add the backfill to the original dataframe but as their own row
df_w_cmpd_out = pd.concat([df_w_cmpd,df_backfill], ignore_index=True)

SyntaxError: invalid syntax (1199672028.py, line 13)

### Load source layouts

In [ ]:
## Load dynamically generated source plate(s)

# For now, only load DMSO source plate
# TODO: Handle multiple solvents if needed
name = 'support-files/colo8-SOURCE-dmso.csv'

df_source = pd.read_csv(name, sep=',')
print(f"Loaded source plate: {name}")
print(f"Total wells in source plate: {len(df_source)}")

Loaded source plate: support-files/colo8-SOURCE-dmso.csv
Total wells in source plate: 20


In [ ]:
df_source['cmpd_w_stock'] = df_source['Compound'] + "[" + df_source['CONCmM'].astype(str)+ "]"

In [ ]:
print(f"✓ Source plate has {len(df_source)} wells")
print(f"  Compounds: {df_source['Compound'].nunique()}")
display(df_source[['well', 'Compound', 'CONCmM']].head(10))

✓ Source plate has 20 wells
  Compounds: 12


,well,Compound,CONCmM
0,A1,abemaciclib,10.0
1,B1,abemaciclib,1.0
2,A2,binimetinib,10.0
3,B2,binimetinib,1.0
4,A3,fluorouracil,10.0
5,B3,fluorouracil,1.0
6,A4,gemcitabine,10.0
7,B4,gemcitabine,1.0
8,A5,olaparib,10.0
9,B5,olaparib,1.0


### generate source plate layouts

the first source plates is filled with compounds at the highest stock concentrations, then every following source plate will be a copy of this plate as long as there are lower stock concentrations. 

since this experiment takes a SelleckChem library, the first source plate layout takes on their layout

In [ ]:
print_echo = pd.DataFrame(columns=['Sample Name', 'Source Plate Name', 'Source well', 'Destination Plate Barcode', 'destination well', 'Transfer Volume', 'Source Plate Type', 'Destination Plate Type', 'Destination Well X Offset', 'Destination Well Y Offset'])

In [ ]:
print(f"✓ Generated protocol with {len(df_w_cmpd_out)} transfers (compounds + backfill)")
print(f"  Compounds: {len(df_w_cmpd)} | DMSO backfill: {len(df_backfill)}")
display(df_w_cmpd_out[['well', 'cmpdname', 'CONCuM', 'CompVol_nL', 'cmpd_w_stock']].head(10))

✓ Generated protocol with 418 transfers (compounds + backfill)
  Compounds: 308 | DMSO backfill: 110


,well,cmpdname,CONCuM,CompVol_nL,cmpd_w_stock
0,B2,trifluridine,3.125,12.5,trifluridine[10.0]
1,B3,etoposide,2.500,10.0,etoposide[10.0]
2,B4,olaparib,3.125,12.5,olaparib[10.0]
3,B5,binimetinib,10.000,40.0,binimetinib[10.0]
4,B6,gemcitabine,1.000,40.0,gemcitabine[1.0]
5,B7,fluorouracil,1.000,40.0,fluorouracil[1.0]
6,B8,abemaciclib,1.000,40.0,abemaciclib[1.0]
7,B9,fluorouracil,3.125,12.5,fluorouracil[10.0]
8,B10,veliparib,1.000,40.0,veliparib[1.0]
9,B11,berberine chloride,2.500,10.0,berberine chloride[10.0]


In [ ]:
# Fill the dataframe with the both compund and source information
print_echo['Sample Name'] = df_w_cmpd_out['cmpd_w_stock']
print_echo['Destination Plate Barcode'] = df_w_cmpd_out['plateID']
print_echo['destination well'] = df_w_cmpd_out['well']
print_echo['Transfer Volume'] = df_w_cmpd_out['CompVol_nL']
print_echo['Source Plate Type'] = '384PP_DMSO2'
print_echo['Destination Plate Type'] = 'Corning_384w_3784'
print_echo['Destination Well X Offset'] = 1050
print_echo['Destination Well Y Offset'] = -1050

# Now for each row in the cmpd_out dataframe, find the location of the cmpd in the source dataframe
for index, row in df_w_cmpd_out.iterrows():
    cmpd = row['cmpd_w_stock']
    # print(cmpd)
    source = df_source[df_source['cmpd_w_stock'] == cmpd]
    # print(source)
    if source.empty:
        print('No source found for compound:', cmpd)
        continue
    source_plate = source['sourceID'].values[0]
    source_well = source['well_source'].values[0]
    print_echo.loc[index, 'Source Plate Name'] = source_plate
    print_echo.loc[index, 'Source well'] = source_well

    # print_echo['Source_plate'] = df_source['plateID']
# print_echo['Source well'] = df_source['well']

In [ ]:
print_echo

,Compound,Source_plate,Source well,Destination plate,destination well,Transfer Volume,Source Plate Type,Destination Plate Type,Destination Well X Offset,Destination Well Y Offset
0,trifluridine[10.0],source_dmso,A07,colo8-v1-VP-organoid-48h-P1-L1,B2,12.5,384PP_DMSO2,Corning_384w_3784,1050,-1050
1,etoposide[10.0],source_dmso,A10,colo8-v1-VP-organoid-48h-P1-L1,B3,10.0,384PP_DMSO2,Corning_384w_3784,1050,-1050
2,olaparib[10.0],source_dmso,A05,colo8-v1-VP-organoid-48h-P1-L1,B4,12.5,384PP_DMSO2,Corning_384w_3784,1050,-1050
3,binimetinib[10.0],source_dmso,A02,colo8-v1-VP-organoid-48h-P1-L1,B5,40.0,384PP_DMSO2,Corning_384w_3784,1050,-1050
4,gemcitabine[1.0],source_dmso,B04,colo8-v1-VP-organoid-48h-P1-L1,B6,40.0,384PP_DMSO2,Corning_384w_3784,1050,-1050
...,...,...,...,...,...,...,...,...,...,...
413,dmso[100.0],source_dmso,A12,colo8-v1-VP-organoid-48h-P1,O17,30.0,384PP_DMSO2,Corning_384w_3784,1050,-1050
414,dmso[100.0],source_dmso,A12,colo8-v1-VP-organoid-48h-P1,O18,27.5,384PP_DMSO2,Corning_384w_3784,1050,-1050
415,dmso[100.0],source_dmso,A12,colo8-v1-VP-organoid-48h-P1,O2,27.5,384PP_DMSO2,Corning_384w_3784,1050,-1050
416,dmso[100.0],source_dmso,A12,colo8-v1-VP-organoid-48h-P1,O23,30.0,384PP_DMSO2,Corning_384w_3784,1050,-1050


In [ ]:
# Export the file
print_echo_filename = 'print_echo_{}'.format(exp_name)

print_echo.to_csv("{}/{}.csv".format(OutputDir,print_echo_filename), index=False, sep=",")

## Experiment Summary Report

In [ ]:
#
# Final Experiment Summary Report
#

from io import StringIO
import sys

# Capture output to both console and string buffer
class TeeOutput:
    def __init__(self):
        self.terminal = sys.stdout
        self.log = StringIO()
    
    def write(self, message):
        self.terminal.write(message)
        self.log.write(message)
    
    def flush(self):
        self.terminal.flush()
    
    def getvalue(self):
        return self.log.getvalue()

# Start capturing output
tee = TeeOutput()
sys.stdout = tee

print("=" * 80)
print("ECHO LIQUID HANDLER PROTOCOL GENERATION COMPLETE")
print("=" * 80)
print()

# Experiment Info
print(f"Experiment Name: {exp_name}")
print(f"Well Volume: {well_vol_uL} µL")
print(f"Max DMSO %: {dmso_max_perc}%")
print()

# Compound Statistics
print("=" * 80)
print("COMPOUND STATISTICS")
print("=" * 80)
unique_compounds = df_w_cmpd['cmpdname'].unique()
total_plates = df_w_cmpd['plateID'].nunique()

# Categorize compounds using cmpdnum (original codes) since cmpdname is mapped to ProductName
# Extract base compound code from cmpdnum (e.g., "ref-001_2.5" -> "ref-001")
df_w_cmpd_temp = df_w_cmpd.copy()
df_w_cmpd_temp['compound_code'] = df_w_cmpd_temp['cmpdnum'].str.split('_').str[0]

unique_codes = df_w_cmpd_temp['compound_code'].unique()
treatment_codes = [c for c in unique_codes if not (c.startswith('ref-') or c == 'dmso')]
positive_control_codes = [c for c in unique_codes if c.startswith('ref-')]
negative_control_codes = [c for c in unique_codes if c == 'dmso']

# Get the actual compound names for each category
treatment_compounds = df_w_cmpd_temp[df_w_cmpd_temp['compound_code'].isin(treatment_codes)]['cmpdname'].unique()
positive_controls = df_w_cmpd_temp[df_w_cmpd_temp['compound_code'].isin(positive_control_codes)]['cmpdname'].unique()
negative_controls = df_w_cmpd_temp[df_w_cmpd_temp['compound_code'].isin(negative_control_codes)]['cmpdname'].unique()

print(f"Total plates: {total_plates}")
print(f"Total unique compounds: {len(unique_compounds)}")
print(f"  - Treatment compounds: {len(treatment_compounds)}")
print(f"  - Positive controls: {len(positive_controls)}")
print(f"  - Negative controls: {len(negative_controls)}")
print(f"Total treatment wells: {len(df_w_cmpd)}")
print()

# Replicate counts with dose information
print("Replicates per compound (wells × concentrations):")
for compound in sorted(df_w_cmpd['cmpdname'].unique()):
    compound_data = df_w_cmpd[df_w_cmpd['cmpdname'] == compound]
    total_wells = len(compound_data)
    n_concentrations = compound_data['CONCuM'].nunique()
    
    if n_concentrations > 1:
        replicates_per_conc = total_wells // n_concentrations
        # Only use ~ if replicates are not evenly distributed
        if total_wells % n_concentrations == 0:
            print(f"  {compound:25s} : {total_wells:3d} wells ({n_concentrations} concentrations, {replicates_per_conc} replicates each)")
        else:
            print(f"  {compound:25s} : {total_wells:3d} wells ({n_concentrations} concentrations, ~{replicates_per_conc} replicates each)")
    else:
        print(f"  {compound:25s} : {total_wells:3d} wells (1 concentration)")
print()

# Concentration summary (handle DMSO differently as it's in %)
print("=" * 80)
print("CONCENTRATION SUMMARY")
print("=" * 80)
conc_summary = df_w_cmpd.groupby(['cmpdname', 'CONCuM', 'stock_unit']).size().reset_index(name='count')
for compound in sorted(conc_summary['cmpdname'].unique()):
    compound_data = conc_summary[conc_summary['cmpdname'] == compound]
    if compound == 'dmso':
        # DMSO is in percentage
        concs = ', '.join([f"{row['CONCuM']}%" for _, row in compound_data.iterrows()])
    else:
        # Other compounds in µM
        concs = ', '.join([f"{row['CONCuM']} µM" for _, row in compound_data.iterrows()])
    print(f"  {compound:25s} : {concs}")
print()

# DMSO Volume Check per Well
print("=" * 80)
print("DMSO VOLUME CHECK (SOLVENT + BACKFILL)")
print("=" * 80)

# Calculate total DMSO per well (compound volumes + backfill)
dmso_per_well = df_w_cmpd.groupby('well')['CompVol_nL'].sum().reset_index()
dmso_per_well.columns = ['well', 'compound_dmso_nL']

# Add backfill volumes
backfill_volumes = df_backfill[['well', 'CompVol_nL']].copy()
backfill_volumes.columns = ['well', 'backfill_nL']

# Merge to get total DMSO
dmso_check = dmso_per_well.merge(backfill_volumes, on='well', how='left')
dmso_check['backfill_nL'] = dmso_check['backfill_nL'].fillna(0)
dmso_check['total_dmso_nL'] = dmso_check['compound_dmso_nL'] + dmso_check['backfill_nL']

# Calculate target DMSO volume (same as maxDMSO calculation)
target_dmso_nL = (dmso_max_perc / 100) * (well_vol_uL * 1000)

# Get unique DMSO volumes
unique_dmso_volumes = dmso_check['total_dmso_nL'].unique()
print(f"Target DMSO per well: {target_dmso_nL:.1f} nL")
print(f"Unique DMSO volumes found: {sorted(unique_dmso_volumes)}")
print(f"Number of unique volumes: {len(unique_dmso_volumes)}")

# Check if all wells have the correct DMSO volume
tolerance = 0.1  # Allow 0.1 nL difference due to rounding
wells_correct = dmso_check[abs(dmso_check['total_dmso_nL'] - target_dmso_nL) <= tolerance]
wells_incorrect = dmso_check[abs(dmso_check['total_dmso_nL'] - target_dmso_nL) > tolerance]

if len(wells_incorrect) == 0:
    print(f"✓ All {len(wells_correct)} wells have correct DMSO volume ({target_dmso_nL:.1f} nL)")
else:
    print(f"⚠ WARNING: {len(wells_incorrect)} wells have incorrect DMSO volume!")
    print("Wells with incorrect DMSO:")
    for _, row in wells_incorrect.iterrows():
        print(f"  {row['well']}: {row['total_dmso_nL']:.1f} nL (expected {target_dmso_nL:.1f} nL)")
print()

# Volume usage per compound × stock concentration
print("=" * 80)
print("VOLUME USAGE PER COMPOUND × STOCK CONCENTRATION")
print("=" * 80)
volume_summary = df_w_cmpd.groupby(['cmpdname', 'stock_conc_mM', 'stock_unit']).agg({
    'CompVol_nL': 'sum',
    'well': 'count'
}).reset_index()
volume_summary.columns = ['Compound', 'Stock_Conc', 'Stock_Unit', 'Total_nL', 'Wells']

for _, row in volume_summary.sort_values('Compound').iterrows():
    compound = row['Compound']
    stock_conc = row['Stock_Conc']
    stock_unit = row['Stock_Unit'].strip()
    total_nl = row['Total_nL']
    wells = row['Wells']
    
    # Format stock concentration with proper units
    if stock_unit == '%':
        stock_str = f"{stock_conc}%"
    else:
        stock_str = f"{stock_conc} {stock_unit}"
    
    print(f"  {compound:25s} [{stock_str:12s}] : {total_nl:7.1f} nL across {wells:3d} wells")
print()

# Source Plate Info
print("=" * 80)
print("SOURCE PLATE LAYOUT")
print("=" * 80)
print(f"Source plate: source_dmso")
print(f"Total wells: {len(df_source)}")
print(f"File: support-files/colo8-SOURCE-dmso.csv")
print()

# Print source plate layout
print("Layout:")
for _, row in df_source.iterrows():
    well = row['well']
    compound = row['Compound']
    conc = row['CONCmM']
    
    # Determine unit based on compound name
    if compound == 'dmso' or compound == 'water':
        unit = '%'
    else:
        unit = 'mM'
    
    print(f"  {well:4s} : {compound:22s} @ {conc:6.1f} {unit}")
print()

# Transfer Statistics
print("=" * 80)
print("ECHO PROTOCOL STATISTICS")
print("=" * 80)
print(f"Total transfers: {len(print_echo)}")
print(f"  - Compound transfers: {len(df_w_cmpd)}")
print(f"  - DMSO backfill transfers: {len(df_backfill)}")
print()

# Volume statistics
print("Transfer volume statistics (nL):")
print(f"  Min: {print_echo['Transfer Volume'].min():.1f}")
print(f"  Max: {print_echo['Transfer Volume'].max():.1f}")
print(f"  Mean: {print_echo['Transfer Volume'].mean():.1f}")
print(f"  Median: {print_echo['Transfer Volume'].median():.1f}")
print()

# CRITICAL VALIDATION: Check all volumes are divisible by 2.5 nL
print("=" * 80)
print("TRANSFER VOLUME VALIDATION (Echo 2.5 nL Requirement)")
print("=" * 80)

unique_volumes = sorted(print_echo['Transfer Volume'].unique())
print(f"Unique transfer volumes: {unique_volumes}")

# Check divisibility by 2.5
non_divisible = [v for v in unique_volumes if (v % 2.5) != 0]

if non_divisible:
    print(f"\n❌ ERROR: {len(non_divisible)} unique volumes are NOT divisible by 2.5 nL!")
    print(f"   Problematic volumes: {non_divisible}")
    
    # Count how many transfers have problematic volumes
    df_problematic = print_echo[print_echo['Transfer Volume'].isin(non_divisible)]
    print(f"   Affected transfers: {len(df_problematic)} out of {len(print_echo)}")
    print("\n⚠ PROTOCOL CANNOT BE RUN ON ECHO LIQUID HANDLER")
else:
    print(f"✓ All {len(unique_volumes)} unique volumes are divisible by 2.5 nL")
    print(f"✓ All {len(print_echo)} transfers are valid for Echo liquid handler")
    print("\n✓ PROTOCOL IS READY FOR ECHO LIQUID HANDLER")
print()

# Output Files
print("=" * 80)
print("OUTPUT FILES GENERATED")
print("=" * 80)
print(f"✓ Echo protocol: {OutputDir}/print_echo_{exp_name}.csv")
print(f"✓ Plate visualization: {OutputDir}/{exp_name}.html")
print(f"✓ Source plate layout: {SupportDir}/colo8-SOURCE-dmso.csv")
print(f"✓ Combined PLAID data: {SupportDir}/{plaid_combined_filename}.csv")
print()
print("=" * 80)
print("READY FOR LIQUID HANDLER")
print("=" * 80)

# Restore stdout and get captured text
sys.stdout = tee.terminal
report_text = tee.getvalue()

# Save report as PDF using matplotlib
try:
    import matplotlib.pyplot as plt
    from matplotlib.backends.backend_pdf import PdfPages
    
    pdf_filename = f"{OutputDir}/report_{exp_name}.pdf"
    
    # Create PDF
    with PdfPages(pdf_filename) as pdf:
        fig = plt.figure(figsize=(8.5, 11))  # Letter size
        
        # Title section with better formatting
        fig.text(0.5, 0.96, "Echo Liquid Handler Protocol Report", 
                fontsize=16, weight='bold', ha='center')
        fig.text(0.5, 0.93, exp_name, 
                fontsize=11, ha='center', family='monospace')
        fig.text(0.5, 0.905, f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", 
                fontsize=9, ha='center', style='italic')
        
        # Add horizontal line
        fig.add_artist(plt.Line2D([0.1, 0.9], [0.89, 0.89], color='black', linewidth=1))
        
        # Add report text
        fig.text(0.05, 0.87, report_text, 
                fontsize=6.5, family='monospace', verticalalignment='top',
                transform=fig.transFigure)
        
        plt.axis('off')
        pdf.savefig(fig, bbox_inches='tight')
        plt.close()
    
    print(f"✓ Saved report PDF: {pdf_filename}")
    
except ImportError:
    print("⚠ Could not save PDF (matplotlib not available)")
except Exception as e:
    print(f"⚠ Error saving PDF: {e}")

ECHO LIQUID HANDLER PROTOCOL GENERATION COMPLETE

Experiment Name: colo8-v1-VP-organoid-48h-P1
Well Volume: 40 µL
Max DMSO %: 0.1%

COMPOUND STATISTICS
Total plates: 1
Total unique compounds: 12
  - Treatment compounds: 8
  - Positive controls: 3
  - Negative controls: 1
Total treatment wells: 308

Replicates per compound (wells × concentrations):
  abemaciclib               :  30 wells (3 concentrations, 10 replicates each)
  berberine chloride        :  10 wells (1 concentration)
  binimetinib               :  30 wells (3 concentrations, 10 replicates each)
  dmso                      :  38 wells (1 concentration)
  etoposide                 :  10 wells (1 concentration)
  fenbendazole              :  10 wells (1 concentration)
  fluorouracil              :  30 wells (3 concentrations, 10 replicates each)
  gemcitabine               :  30 wells (3 concentrations, 10 replicates each)
  olaparib                  :  30 wells (3 concentrations, 10 replicates each)
  sn-38                